In [ ]:
sc.version
val conf = sc.hadoopConfiguration
val fs = org.apache.hadoop.fs.FileSystem.get(conf)
val hdfshome = fs.getHomeDirectory
println(sc.version)
println(hdfshome)
val files = fs.listFiles(hdfshome, false)

In [ ]:
val rootDir = "/user/fsainz/data/in"
val files = fs.listFiles(new org.apache.hadoop.fs.Path(rootDir), false)

In [ ]:
val rootDir = "hdfs://spark-clu-m/user/fsainz/data/out/"
val files = fs.listFiles(new org.apache.hadoop.fs.Path(rootDir), false)


In [ ]:
fs.exists(new org.apache.hadoop.fs.Path("hdfs://spark-clu-m/user/fsainz/data/in/a02.dat"))


## Split in windows


In [ ]:
import java.io._                                                        
import scala.collection.mutable.ListBuffer   //bE CAREFULL With java imports, better before scala.

val WINDOW = 40;
val STEP = 2;
val TRsamples = 200000;
val scale = 1/200.0;
val inFileName = "a02.dat"

val inputPath =  "/home/fsainz/src/data/" //local filesystem
val fileName = inputPath + inFileName 
val hdfsOutPath = "/user/fsainz/data/out/"
val outFileName = hdfsOutPath + "dict-" + inFileName.split('.')(0) + "_W" + WINDOW +".model"


println("InFile: " + fileName)
println("Ouput Model: " + outFileName)

In [ ]:


// Till Now start reading local file   TRaining file is a02.dat training and run

var inFile =  new File(fileName)
var InFileStream = new FileInputStream( inFile )
val input : DataInputStream  = new DataInputStream(InFileStream)


// val nSamples =  (inFile.length() / 2)   // We are going to read short: 2 bytes..


val nWindows = TRsamples / WINDOW
println("nWindows = " + nWindows.toString)

val buffer = new Array[Double](WINDOW)
var windows = scala.collection.mutable.ListBuffer.empty[Array[Double]]

var pos = 0
var available = 0
var dataWindow = new Array[Double](WINDOW)

for (  w  <- 1 to nWindows.toInt){
    // new object is neeed to append at the end!!
  dataWindow = new Array[Double](WINDOW) 
  var newWin:Boolean = false
  while( ! newWin) {
    val value = input.readShort()
    buffer( pos ) = (value.toDouble * scale) 
    available += 1
    pos +=1
    if ( pos == WINDOW )  // buffer ends
      pos = 0  
    if (available == WINDOW) {    
      var j =0
      for( i <- pos until WINDOW){
         dataWindow(j) = buffer(i)
         j +=1
      }
      for( i <- 0 until pos ){
         dataWindow(j) = buffer(i)
         j +=1
      }
      //dataWindow.foreach(print)
      //println("|")
      newWin = true
      available = WINDOW-STEP   // ready for read windows displacement offset 
    }
  }
  windows.append(dataWindow  )  
}
input.close()

In [ ]:
nWindows*WINDOW*2

## Defining windowing function

In [ ]:

var windowingF  =  new Array[Double](WINDOW)   //differs of Array[Double](WINDOW) uajj
for ( i <- 0 until WINDOW) {
    val y = Math.sin(Math.PI * i / (WINDOW - 1.0));
          windowingF(i)=  y * y
 }


In [ ]:
val ECGRdd = sc.parallelize(windows)


In [ ]:
// spark 2.1 mllib
import org.apache.spark.mllib.clustering.{KMeans, KMeansModel}
import org.apache.spark.mllib.linalg.Vectors
import org.apache.spark.mllib.regression.LabeledPoint 

In [ ]:
val ECGwnRdd = ECGRdd.map( x => ((x,windowingF).zipped.map(_*_))).map(x =>( x, Vectors.norm( Vectors.dense(x),2) )).map(x => Vectors.dense(x._1.map( c => c/ x._2)) ) 
ECGwnRdd.count
ECGwnRdd.first

In [ ]:
    ECGwnRdd.cache()  // important to kmeans time
    val numIterations = 20// we want to assure a global mininum with 20 it seem not to be reached
    val numClusters = 400
    val clusters = KMeans.train(ECGwnRdd, numClusters, numIterations)

In [ ]:
clusters

In [ ]:
clusters.save(sc, outFileName)
println(outFileName + " SAVED")

## Try Binay
And improvement could be read file as binary file.
/ TODO:
//How to read a binary file from spark, that is partioned...

In [ ]:
val hdfsinputPath =  "/user/fsainz/data/in/"
val fileName = hdfsinputPath + inFileName
val rawData = sc.binaryRecords(fileName,WINDOW)

In [ ]:
val n = rawData.count
println(n)

In [ ]:
rawData.first.getClass.getSimpleName